In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceHub
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms import Ollama

import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# Read the pdfs from the folder
loader = PyPDFDirectoryLoader("./us_census")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs)

In [4]:
# Embedding
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)


/Users/lucamanca/Codes/AIAgent/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# VectorStore Creation
vectorstore = FAISS.from_documents(final_documents[:120], huggingface_embeddings)

In [6]:
# Query using Similarity Search
query = "What is the health insurance cover?"
relevant_docments = vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [7]:
# Retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x156d51eb0> search_kwargs={'k': 3}


In [12]:
# Hugging Face LLM
hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature": 0.1, "max_length": 500}
)
query = "What is the health insurance coverage?"
#hf.invoke(query)

/Users/lucamanca/Codes/AIAgent/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [13]:
# Hugging Face models can be run locally through the HuggingFacePipeline class.
#hf = HuggingFacePipeline.from_model_id(
#    model_id="mistralai/Mistral-7B-v0.1",
#    task="text-generation",
#    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
#)

#llm = hf
#llm.invoke(query)

In [14]:
# Load Ollama LAMA2 LLM model
llm = Ollama(model="llama2")

In [17]:
# RAG pipeline
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context.

{context}
Question:{question}

Helpful Answers:
"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"])
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [18]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

Based on the provided context, the answer to the question "What are the differences in uninsured rates by state in 2022?" is:

In 2022, the uninsured rate ranged from 2.4% in Massachusetts to 16.6% in Texas across states in the United States. Medicaid coverage accounted for a portion of the difference in uninsured rates between states that expanded Medicaid eligibility and those that did not. From 2021 to 2022, uninsured rates decreased across 27 states, while only Maine had an increase. The uninsured rate in Maine increased from 5.7% to 6.6%, although it remained below the national average.

In terms of private coverage rates, they were not statistically different in North Dakota and Utah in 2022.
